In [24]:
import os
import yaml
from rich import print

from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import SequentialChain, LLMChain

## Step 0: Front-camera view photo
Use `image_url` to import the photo you want to test. 

Feel free to try new photo with your own link!

In [62]:
from gradio_client import Client
from IPython.display import Image

image_url = "https://github.com/PJLab-ADG/DriveLikeAHuman/blob/main/assets/cones_on_truck_1.jpg?raw=true"
Image(url= image_url)


## Step 1: Use LLaMA-Adapter to generate a description of the photo

In [57]:
client = Client("http://llama-adapter.opengvlab.com/")
llama_result = client.predict(
    image_url,  # str representing input in 'Input' Image component
    "Describe the picture as details as possible and focus on the main object.Do not describe what you don't see. The objects in the picture are moving.",    
    512,  # int | float representing input in 'Max length' Slider component
    0.1,  # int | float representing input in 'Temperature' Slider component
    0.75,  # int | float representing input in 'Top p' Slider component
    fn_index=1
)
llama_result

Loaded as API: http://llama-adapter.opengvlab.com/ ✔


"The image features a white truck driving down a street, carrying a large load of traffic cones. The truck is filled with numerous traffic cones, which are stacked and secured in the back of the vehicle. The cones are of various sizes and are placed in different positions, covering the entire length of the truck. The scene captures the truck's journey, showcasing the impressive amount of traffic cones it is carrying."

## Step 2: Load  LLM and format LLaMA-Adapter output
The code support both OpenAI API or Azure OpenAI service. 

Set up your API key in `config.yaml`.

In [58]:
OPENAI_CONFIG = yaml.load(open('config.yaml'), Loader=yaml.FullLoader)

if OPENAI_CONFIG['OPENAI_API_TYPE'] == 'azure':
    os.environ["OPENAI_API_TYPE"] = OPENAI_CONFIG['OPENAI_API_TYPE']
    os.environ["OPENAI_API_VERSION"] = OPENAI_CONFIG['AZURE_API_VERSION']
    os.environ["OPENAI_API_BASE"] = OPENAI_CONFIG['AZURE_API_BASE']
    os.environ["OPENAI_API_KEY"] = OPENAI_CONFIG['AZURE_API_KEY']
    llm = AzureChatOpenAI(
        deployment_name=OPENAI_CONFIG['AZURE_MODEL'],
        temperature=0,
        max_tokens=1024
    )
elif OPENAI_CONFIG['OPENAI_API_TYPE'] == 'openai':
    os.environ["OPENAI_API_KEY"] = OPENAI_CONFIG['OPENAI_KEY']
    llm = ChatOpenAI(
        temperature=0,
        max_tokens=1024
    )


In [59]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="""
            Here is a detailed description of a image, you need to summarize them into several key points. 
            The format is:
            1. [key point 1]
            2. [key point 2]
            ...
            Here is the  description:
            {llama_results}?
            """,
            input_variables=["llama_results"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt_template)

observation_result = chain.run(llama_result)
print(observation_result)

1. A white truck is driving down a street.
2. The truck is carrying a large load of traffic cones.
3. The cones are of various sizes and are stacked and secured in the back of the vehicle.
4. The cones cover the entire length of the truck.
5. The scene showcases the impressive amount of traffic cones the truck is carrying.

## Step 3: Find the unusual part of this driving scenario

In [60]:
observation_result += "\nAnd ego car is driving behind them and maintains a proper distance."

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="""
        You are a driving assitant. You are designed to assist the vehicle `ego` in making driving decisions based on scenario files provided by humans. The scenario file is passed in as a json file describing the structure of the road network and the state of the vehicles on the road network. You are very strict to the description of the file and does not falsify the names of the roads and vehicles, nor the state information of the roads and vehicles. \
        You understand the state of the vehicle `ego` and suggests possible actions for the vehicle based on the state. Further, you can evaluate the actions proposed in the previous step and thus select the most appropriate action. \
        In the following, you will be given a scenario file in json format, and also some observations from the sensors on the vehicle. \
        You only need to make inferences based on the available information. You do not have to assume or consider dangers that have NOT occurred.\
        You need to analyze the scenario step by step and tell me is there anything worth noting or unusual in this driving scenario. \
        You should NOT assume scenarios that are not happening, but only for the current observation.\
        Here is the observation from the sensors: ```{observation}```
            """,
        input_variables=["observation"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt_template)

abnormal_situation = chain.run(observation_result)
print(abnormal_situation)

Based on the given observation, there is nothing particularly unusual or worth noting in this driving scenario. The
vehicle `ego` is simply driving behind a white truck that is carrying a large load of traffic cones, which are 
stacked and secured in the back of the vehicle. The cones cover the entire length of the truck, and the scene 
showcases the impressive amount of traffic cones the truck is carrying. The vehicle `ego` is maintaining a proper 
distance from the truck, which is a safe driving practice.

## Step 4: Make the final decision

In [61]:
second_prompt = ChatPromptTemplate.from_template(
    """
    You are a mature and experienced driver on the road, you are good at handling complex and unusual traffic scenarios. You've seen a lot corner cases when driving.\
    You can balance very well between safety and efficiency of driving. \
    You need to make inferences based on the available information. \
    Here is something worth noting in current scenario: ```{abnormal_text}``` \
    Here is the observation from current sensors: ```{observation}```\
    Now you should:
    1. Summary the scenario and reason whether the current scenario dangerous or not.\
    2. Should the driver on 'ego' decide to decelarate the car because of current situation? You should answer yes or no and then explain why.\
    """
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="analyze")
overall_simple_chain = SequentialChain(chains=[chain_two],
                                                input_variables=["abnormal_text", "observation"],
                                                output_variables=["analyze"],
                                                )
response = overall_simple_chain({"abnormal_text":f"""{abnormal_situation}""", "observation":f"""{observation_result}"""})
print(response["analyze"])


1. Summary: The current scenario involves a white truck carrying a large load of traffic cones, which cover the 
entire length of the truck. The vehicle 'ego' is driving behind the truck and maintaining a safe distance. There is
nothing particularly dangerous or unusual in this scenario.

2. No, the driver on 'ego' should not decide to decelerate the car because of the current situation. The vehicle 
'ego' is already maintaining a proper distance from the truck, which is a safe driving practice. Additionally, 
there is nothing particularly dangerous or unusual in this scenario that would require the driver to slow down.